In [ ]:
%load_ext autoreload
%autoreload 2

Declaration of parameters (you must also add a tag for this cell - parameters)

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={   
    "threshold_iou" : 0.5,
    "threshold_MAp05": 0.01
}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(    
    tmp_inputs =
    [
        { ENTITY_NAME: "coco_test_dataset" }, # ground-true test dataset
        { ENTITY_NAME: "coco_inference_result_dataset" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

### Load coco_test_dataset and coco_inference_result_dataset

In [ ]:
import os.path as osp
from utils.coco import load_coco_file

tmp_inputs = substep.tmp_inputs()

# reading ground-truth test dataset 
test_coco_dataset_path = osp.join(osp.join(tmp_inputs.coco_test_dataset, "test_coco_annotations.json"))

# reading inference detect result dataset 
eval_coco_dataset_file = osp.join(tmp_inputs.coco_inference_result_dataset, 'eval_coco_dataset.json')
eval_coco_dataset = load_coco_file(eval_coco_dataset_file)

### Evaluate the test dataset 

In [ ]:
iou_type = eval_coco_dataset['info'].get('iouType', 'bbox')

#### Eval Precision-Recall Curve

In [ ]:
from pycocotools.coco import COCO
from utils.extra.curves import Curves
from utils.coco.encoder import encode_dict_to_numpy

eval_annotations = encode_dict_to_numpy(eval_coco_dataset['annotations'])

test_coco_obj = COCO(test_coco_dataset_path)
eval_coco_obj = test_coco_obj.loadRes(eval_annotations)

curves = Curves(
    test_coco_obj,
    eval_coco_obj,
    iou_tresh=substep_params['threshold_iou'], 
    recall_count=1000, 
    iou_type=iou_type
)

_curve = curves.build_curve('category')
curves.plot_pre_rec(curves=_curve, plotly_backend=True)

#### Eval Average Precision, Recall Metrics

In [ ]:
from pycocotools.cocoeval import COCOeval

# running evaluation
coco_eval = COCOeval(test_coco_obj, eval_coco_obj, iou_type)
coco_eval.params.imgIds  = sorted(test_coco_obj.getImgIds())
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

### Check weigths_pth by metric mAP@0.5 IoU

In [ ]:
threshold_MAp05 = substep_params["threshold_MAp05"]

map_05 = coco_eval.stats[1]
print(f"mAP@0.5 = {map_05}")
assert map_05 > threshold_MAp05, f"The calculated MAp metric on the test dataset is less than the acceptable value <{threshold_MAp05}"